# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#HK1: Obtained all dependencies from second Assignment

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import  TabularDatasetFactory

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#HK2: Get the workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-final-project'

experiment=Experiment(ws, experiment_name)

# start logging
run = experiment.start_logging


In [3]:
#HK3: Get the Cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "assign3-cluster1"
amlcompute_cluster_name = "notebook195333"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Found existing cluster, use it.


In [4]:
#HK4: Download the data into Azure; Create a dataset

#data_location = "https://github.com/AI-Treasure/Capstone-3rd-assignment/heart.csv"
#data_location = "./heart.csv"
#ds = TabularDatasetFactory.from_delimited_files(path=data_location)

data = pd.read_csv("./heart.csv")
df = pd.DataFrame(data)

if not os.path.isdir('data'):
    os.mkdir('data')

df.to_csv("data/heart.csv", index=False)
ds = ws.get_default_datastore()
ds.upload(src_dir='./data',overwrite=True, show_progress=True)
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('heart.csv'))
train_data = train_data.register(workspace=ws,name='heart',description='test')


"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [5]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    max_concurrent_iterations=4,
    enable_early_stopping=True,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='HeartDisease',
    n_cross_validations=2)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on notebook195333 with default configuration
Running on remote compute: notebook195333


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-final-project,AutoML_91137c01-ebcd-4cab-8f38-7a5efcb132f1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-final-project,AutoML_91137c01-ebcd-4cab-8f38-7a5efcb132f1,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_91137c01-ebcd-4cab-8f38-7a5efcb132f1',
 'target': 'notebook195333',
 'status': 'Completed',
 'startTimeUtc': '2022-05-11T11:09:48.60095Z',
 'endTimeUtc': '2022-05-11T12:01:11.446753Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'notebook195333',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-final-project","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-195333","workspace_name":"quick-starts-ws-195333","region":"southcentralus","compute_target":"notebook195333","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# HK-Step 6: Retrieve and save your best automl model.

best_run_auto, best_fit_model = remote_run.get_output()

print(best_run_auto)


print("best-run details :", best_run_auto.get_details())
print('\n')
print("best-run id : ",best_run_auto.id)
print('\n')
print("best-run file names :", best_run_auto.get_file_names())
print('\n')
print("best-run metrics :", best_run_auto.get_metrics())



Run(Experiment: capstone-final-project,
Id: AutoML_91137c01-ebcd-4cab-8f38-7a5efcb132f1_36,
Type: azureml.scriptrun,
Status: Completed)
best-run details : {'runId': 'AutoML_91137c01-ebcd-4cab-8f38-7a5efcb132f1_36', 'target': 'notebook195333', 'status': 'Completed', 'startTimeUtc': '2022-05-11T11:54:52.449342Z', 'endTimeUtc': '2022-05-11T12:01:10.185922Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-final-project\',\'compute_target\':\'notebook195333\',\'subscription_id\':\'f5091c60-1c3c-430f-8d81-d802f6bf2414\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_ru

In [10]:
#HK7: Save the best model
import joblib

joblib.dump(best_fit_model, 'best_fit_model.pkl')

['best_fit_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
#HK8: Register the model
from azureml.core.model import Model

model = Model.register(workspace = ws, model_name = 'HK_best_fit_automl_model', model_path = 'best_fit_model.pkl')


Registering model HK_best_fit_automl_model


In [12]:
#HK9: Deploy the Model

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_run_auto.get_environment()


inference_config = InferenceConfig(entry_script='./score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "heartservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-11 12:38:01+00:00 Creating Container Registry if not exists..
2022-05-11 12:48:02+00:00 Registering the environment.
2022-05-11 12:48:02+00:00 Use the existing image.
2022-05-11 12:48:02+00:00 Generating deployment configuration.
2022-05-11 12:48:04+00:00 Submitting deployment to compute.
2022-05-11 12:48:06+00:00 Checking the status of deployment heartservice..
2022-05-11 12:50:57+00:00 Checking the status of inference endpoint heartservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://74674048-6d71-4e42-bdbc-7c0ba5a0c445.southcentralus.azurecontainer.io/score
http://74674048-6d71-4e42-bdbc-7c0ba5a0c445.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
#HK10: Request the web service

import requests
import json

# URL for the web service

#scoring_uri = 'http://b1f8e07e-07fc-420e-b337-2689beda1093.southcentralus.azurecontainer.io/score'
scoring_uri = 'http://74674048-6d71-4e42-bdbc-7c0ba5a0c445.southcentralus.azurecontainer.io/score'
# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "Age":40, 
                "Sex":'M', 
                "ChestPainType":'ATA', 
                "RestingBP":140, 
                "Cholesterol":289, 
                "FastingBS":0,
                "RestingECG":'Normal', 
                "MaxHR":172, 
                "ExerciseAngina":0, 
                "Oldpeak":0.0, 
                "ST_Slope":'Up'}
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [0]


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
